In [1]:
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
import numpy as np
import pickle
from torch.utils import data
import torch
import random
import matplotlib.pyplot as plt
from os import path

In [7]:
''' for resizing images
import cv2

def resize_and_save_images(loc, original_dir, saving_dir, height = 160, width = 160):
    data = pd.read_csv(loc)
    for i in range(len(data)):
        image_filename = f'{data.iloc[i].id_code.strip()}.png'
        file_loc = original_dir + image_filename
        saving_loc = saving_dir + image_filename
        if not path.exists(saving_loc):
            try:
                img = cv2.imread(file_loc)
                img = cv2.resize(img, (width, height))
                cv2.imwrite(saving_loc, img)
            except:
                unsaved.append(f'{image_filename}')
                # print(f'{image_filename}')
                
resize_and_save_images('/u/home/r/rosemary/scratch/train.csv', 
                       '/u/home/r/rosemary/scratch/train_images_resized/', 
                       '/u/home/r/rosemary/scratch/train_images_smol/')
'''

In [2]:
def get_image_label(data, dir, indices):

    # just looking at 32 examples for testing purposes
    X = np.array([np.asarray(Image.open(f'{dir}{data.iloc[i].id_code}.png')) for i in indices])
    y = np.array([int(data.iloc[i].diagnosis) for i in indices])

    return X,y

def get_dataset(path, dir, train_size = 0.8, valid_size = 0.1):

    data = pd.read_csv(path)
    arr = [i for i in range(len(data))]
    random.shuffle(arr)
    train_indices = arr[: int(train_size * len(data)) ]
    train_X, train_y = get_image_label(data, dir, train_indices)
    train_X = train_X / 255.0

    valid_indices = arr[int(train_size * len(data)) : int((train_size + valid_size) * len(data))]
    valid_X, valid_y = get_image_label(data, dir, valid_indices)
    valid_X = valid_X / 255.0

    test_indices = arr[int((train_size + valid_size) * len(data)) : ]
    test_X, test_y = get_image_label(data, dir, test_indices)
    test_X = test_X / 255.0

    return train_X, train_y, valid_X, valid_y, test_X, test_y


In [3]:
train_X, train_y, valid_X, valid_y, test_X, test_y = get_dataset("/u/home/r/rosemary/scratch/train.csv", 
                                                                 '/u/home/r/rosemary/scratch/train_images_smol/')
train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape

((2929, 160, 160, 3),
 (2929,),
 (366, 160, 160, 3),
 (366,),
 (367, 160, 160, 3),
 (367,))

In [4]:
class DataGenerator(Dataset):
    def __init__(self, X, y):
        super(DataGenerator, self).__init__()
        self.X = np.transpose(X, (0, 3, 1, 2))
        self.y = y
        self.length = len(X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.length

In [5]:
bsz = 128

train_dataset = DataGenerator(train_X, train_y)
train_loader = data.DataLoader(train_dataset, batch_size= bsz, shuffle = True)

valid_dataset = DataGenerator(valid_X, valid_y)
valid_loader = data.DataLoader(valid_dataset, batch_size = bsz, shuffle = True)

test_dataset = DataGenerator(test_X, test_y)
test_loader = data.DataLoader(test_dataset, batch_size = bsz, shuffle = False)

In [6]:
class generate_model(torch.nn.Module):

    def __init__(self, base_model, hidden = 128, num_outs = 5):
        super(generate_model, self).__init__()

        # create a dummy input
        dummy_input = torch.rand(1, 3, 320, 320)
        out = base_model(dummy_input.to(device).float())
        input_size = out.shape[1]

        self.base_model = base_model
        self.fc = torch.nn.Sequential(
                                torch.nn.Linear(input_size, hidden), 
                                torch.nn.ReLU(),
                                torch.nn.Linear(hidden, num_outs)
                                )

    def forward(self, x):
        x = self.base_model(x)
        pred = self.fc(x)
        return pred

In [7]:
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
print(device)

cpu


In [16]:
basemodel = models.vgg16().to(device)
model = generate_model(base_model = basemodel).to(device)

model_file = "vgg16.pt"

lr = 1e-4

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [17]:
# from torchsummary import summary
# from pytorch_model_summary import summary

# summary(model, torch.zeros(1,3, 160, 160).to(device).float(), show_input=True)

In [18]:
def evaluate(model, objective, loader):

    model.eval()
    total_loss = 0
    size = 0
    
    with torch.no_grad():
        for batch_idx, data_batch in enumerate(loader):

            X, y = data_batch[0].to(device).float(), data_batch[1].to(device)
            # X, y = map(lambda t: t.to(device).float(), (X, y))

            prediction = model(X)
            total_loss += objective(prediction, y) * X.shape[0]
            size += X.shape[0]

    total_loss = total_loss / size

    return total_loss

def train(model, objective, optimizer, train_loader, valid_loader, epochs = 1, save_interval = 1, patience = 3):
  
    model.train()

    val_loss = 1e7
    pat = patience

    for epoch in range(1, epochs + 1):
        train_loss = 0
        size = 0

        for batch_idx, data_batch in enumerate(train_loader):

            optimizer.zero_grad()

            train_X, train_y = data_batch[0].to(device).float(), data_batch[1].to(device)
            # train_X, train_y = map(lambda t: t.to(device).float(), (train_X, train_y))

            prediction = model(train_X)
            loss = objective(prediction, train_y)
            loss.backward()

            train_loss += loss.item() * train_X.shape[0]
            size += train_X.shape[0]

            optimizer.step()

        avg_loss = train_loss / size

        rt_val_loss = evaluate(model, objective, valid_loader)
        model.train()

        print(f'Epoch {epoch}: Training Loss : {avg_loss} | Validation loss : {rt_val_loss}')

        if rt_val_loss < val_loss:
            val_loss = rt_val_loss
            torch.save(model.state_dict(), model_file)
            pat = patience
        else:
            pat = pat - 1
            if pat == 0:
                print('Training Complete --> Exiting')
                break    

In [ ]:
train(model, criterion, optimizer, train_loader, valid_loader, epochs = 50)

Epoch 1: Training Loss : 1.3655645506664444 | Validation loss : 1.2529411315917969
Epoch 2: Training Loss : 1.0843689627271336 | Validation loss : 1.0093023777008057
Epoch 3: Training Loss : 0.9679289818542652 | Validation loss : 0.930590033531189
Epoch 4: Training Loss : 0.874453433872938 | Validation loss : 0.8248466849327087
Epoch 5: Training Loss : 0.820864018908781 | Validation loss : 0.8088432550430298
Epoch 6: Training Loss : 0.8067173152785531 | Validation loss : 0.8106438517570496
Epoch 7: Training Loss : 0.7813758129103268 | Validation loss : 0.832765519618988
Epoch 8: Training Loss : 0.7958435782722012 | Validation loss : 0.7808632850646973


In [ ]:
model = torch.load_state_dict(torch.load(model_file))
evaluate(model, criterion, test_loader)